Notebook written by [Zhedong Zheng](https://github.com/zhedongzheng)

![alt text](../img/conve.png)

In [1]:
from tqdm import tqdm
import tensorflow as tf
import numpy as np
import sklearn
import pprint
import itertools
import os
import sys

sys.path.append(os.path.dirname(os.getcwd()))
from data import WN18

In [2]:
class Config:
    n_epochs = 100
    batch_size = 150
    embed_dim = 200

In [3]:
"""
e: entity
s: subject
p: predicate
o: object
"""
def glance_dict(d, n=5):
    return dict(itertools.islice(d.items(), n))


def read_triples(path):
    triples = []
    with open(path, 'rt') as f:
        for line in f.readlines():
            s, p, o = line.split()
            triples += [(s.strip(), p.strip(), o.strip())]
    return triples


def load_triple():
    WN18.download()
    triples_tr = read_triples('../data/WN18/wn18/train.txt')
    triples_va = read_triples('../data/WN18/wn18/valid.txt')
    triples_te = read_triples('../data/WN18/wn18/test.txt')
    triples_all = triples_tr + triples_va + triples_te
    return triples_all, triples_tr, triples_va, triples_te


def build_vocab(triples):
    params = {}
    e_set = {s for (s, p, o) in triples} | {o for (s, p, o) in triples}
    p_set = {p for (s, p, o) in triples}
    params['e_vocab_size'] = len(e_set)
    params['p_vocab_size'] = len(p_set)
    e2idx = {e: idx for idx, e in enumerate(sorted(e_set))}
    p2idx = {p: idx for idx, p in enumerate(sorted(p_set))}
    return e2idx, p2idx, params


def build_multi_label(triples_tr):
    sp2o = {}
    for (_s, _p, _o) in triples_tr:
        s, p, o = e2idx[_s], p2idx[_p], e2idx[_o] 
        if (s,p) not in sp2o:
            sp2o[(s,p)] = [o]
        else:
            if o not in sp2o[(s,p)]:
                sp2o[(s,p)].append(o)
    return sp2o


def get_y(triples_tr, e2idx, p2idx, sp2o):
    y = []
    for (_s, _p, _o) in triples_tr:
        s, p, o = e2idx[_s], p2idx[_p], e2idx[_o] 
        temp = np.zeros([len(e2idx)])
        temp[sp2o[(s,p)]] = 1.
        y.append(temp)
    y = np.asarray(y)
    return y


def next_train_batch(triples_tr, e2idx, p2idx, sp2o):
    for i in range(0, len(triples_tr), Config.batch_size):
        _triples_tr = triples_tr[i: i+Config.batch_size]
        x_s = np.asarray([e2idx[s] for (s, p, o) in _triples_tr], dtype=np.int32)
        x_p = np.asarray([p2idx[p] for (s, p, o) in _triples_tr], dtype=np.int32)
        y = get_y(_triples_tr, e2idx, p2idx, sp2o)
        yield (x_s, x_p, y)


def train_input_fn(triples_tr, e2idx, p2idx, s2p2o):
    dataset = tf.data.Dataset.from_generator(
        lambda: next_train_batch(sklearn.utils.shuffle(triples_tr),
                                 e2idx,
                                 p2idx,
                                 s2p2o),
        (tf.int32, tf.int32, tf.float32),
        (tf.TensorShape([None]),
         tf.TensorShape([None]),
         tf.TensorShape([None, len(e2idx)])))
    iterator = dataset.make_one_shot_iterator()
    x_s, x_p, y = iterator.get_next()
    return {'s': x_s, 'p': x_p}, y

In [4]:
def o_next_batch(eval_triples,
                 e2idx, 
                 p2idx):
    for (s, p, o) in tqdm(eval_triples, total=len(eval_triples), ncols=70):
        s_idx, p_idx = e2idx[s], p2idx[p]
        yield np.atleast_1d(s_idx), np.atleast_1d(p_idx)


def o_input_fn(eval_triples,
               e2idx, 
               p2idx):
    dataset = tf.data.Dataset.from_generator(
        lambda: o_next_batch(eval_triples,
                             e2idx, 
                             p2idx),
        (tf.int32, tf.int32),
        (tf.TensorShape([None,]),
         tf.TensorShape([None,])))
    iterator = dataset.make_one_shot_iterator()
    s, p = iterator.get_next()
    return {'s': s, 'p': p}


def evaluate_rank(model,
                  valid_triples,
                  test_triples,
                  all_triples,
                  e2idx,
                  p2idx):
    for eval_name, eval_triples in [('test', test_triples)]:
        _scores_o = list(model.predict(
            lambda: o_input_fn(eval_triples,
                               e2idx, 
                               p2idx)))
        ScoresO = np.reshape(_scores_o, [len(eval_triples), len(e2idx)])
        ranks_o, filtered_ranks_o = [], []
        for ((s, p, o), scores_o) in tqdm(zip(eval_triples, ScoresO),
                                          total=len(eval_triples),
                                          ncols=70):
            s_idx, p_idx, o_idx = e2idx[s], p2idx[p], e2idx[o]
            ranks_o += [1 + np.argsort(np.argsort(- scores_o))[o_idx]]
            filtered_scores_o = scores_o.copy()
            rm_idx_o = [e2idx[fo] for (fs, fp, fo) in all_triples if fs == s and fp == p and fo != o]
            filtered_scores_o[rm_idx_o] = - np.inf
            filtered_ranks_o += [1 + np.argsort(np.argsort(- filtered_scores_o))[o_idx]]
        for setting_name, setting_ranks in [('Raw', ranks_o), ('Filtered', filtered_ranks_o)]:
            mean_rank = np.mean(1 / np.asarray(setting_ranks))
            print('[{}] {} MRR: {}'.format(eval_name, setting_name, mean_rank))
            for k in [1, 3, 5, 10]:
                hits_at_k = np.mean(np.asarray(setting_ranks) <= k) * 100
                print('[{}] {} Hits@{}: {}'.format(eval_name, setting_name, k, hits_at_k))

In [5]:
def forward(features, mode, params):
    batch_sz = tf.shape(features['s'])[0]
    is_training = (mode == tf.estimator.ModeKeys.TRAIN)
    
    e_embed = tf.get_variable('e_embed',
                              [params['e_vocab_size'], Config.embed_dim],
                              initializer=tf.variance_scaling_initializer())
    p_embed = tf.get_variable('p_embed',
                              [params['p_vocab_size'], Config.embed_dim],
                              initializer=tf.variance_scaling_initializer())
    
    s = tf.nn.embedding_lookup(e_embed, features['s'])
    p = tf.nn.embedding_lookup(p_embed, features['p'])
    
    x = tf.concat([tf.reshape(s, [batch_sz, 10, 20, 1]),
                   tf.reshape(p, [batch_sz, 10, 20, 1]),], axis=1)
    x = tf.layers.batch_normalization(x, training=is_training)
    x = tf.layers.dropout(x, 0.2, training=is_training)
    
    x = tf.layers.conv2d(x, 32, (3,3), activation=tf.nn.relu)
    x = tf.layers.flatten(x)
    x = tf.layers.batch_normalization(x, training=is_training)
    x = tf.layers.dropout(x, 0.2, training=is_training)
    
    x = tf.layers.dense(x, Config.embed_dim, tf.nn.relu)
    x = tf.layers.batch_normalization(x, training=is_training)
    x = tf.layers.dropout(x, 0.2, training=is_training)
    
    logits = tf.matmul(x, e_embed, transpose_b=True)
    return logits
    
    
def model_fn(features, labels, mode, params):
    logits = forward(features, mode, params)
    
    if mode == tf.estimator.ModeKeys.TRAIN:
        tf.logging.info('\n'+pprint.pformat(tf.trainable_variables()))
        tf.logging.info('params: %d'%count_train_params())
        
        global_step = tf.train.get_global_step()
        
        loss_op = tf.reduce_sum(tf.nn.sigmoid_cross_entropy_with_logits(logits=logits,
                                                                        labels=labels))
        
        with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)):
            train_op = tf.train.AdamOptimizer().minimize(loss_op,
                                                         global_step = global_step)
        
        return tf.estimator.EstimatorSpec(mode = mode,
                                          loss = loss_op,
                                          train_op = train_op)
    
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode, predictions = tf.sigmoid(logits))


def count_train_params():
    return np.sum([np.prod([d.value for d in v.get_shape()]) for v in tf.trainable_variables()])

In [6]:
triples_all, triples_tr, triples_va, triples_te = load_triple()
e2idx, p2idx, params = build_vocab(triples_all)
sp2o = build_multi_label(triples_tr)

model = tf.estimator.Estimator(model_fn,
                               params = params)

for n_epoch in range(Config.n_epochs):
    tf.logging.info('Epoch %d'%(n_epoch+1))
    model.train(lambda: train_input_fn(triples_tr, e2idx, p2idx, sp2o))
    evaluate_rank(model,
                  triples_va,
                  triples_te,
                  triples_all,
                  e2idx,
                  p2idx,)

Files Already Downloaded
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/sx/fv0r97j96fz8njp14dt5g7940000gn/T/tmpj1bm9lou', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x10aafb710>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Epoch 1
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:
[<tf.Variable 'e_embed:0' shape=(40943, 200) dtype=float32_ref>,
 <tf.Variable 'p_embed:0' shape=(18, 200) dtype=float32_ref>,
 <tf.Variable 'batch_normalization/gamma:0' shape=(1,) dtype=float32_ref

100%|█████████████████████████████| 5000/5000 [01:47<00:00, 46.66it/s]


[test] Raw MRR: 0.012630274747911917
[test] Raw Hits@1: 0.5
[test] Raw Hits@3: 1.28
[test] Raw Hits@5: 1.82
[test] Raw Hits@10: 2.44
[test] Filtered MRR: 0.012637126006984856
[test] Filtered Hits@1: 0.5
[test] Filtered Hits@3: 1.28
[test] Filtered Hits@5: 1.82
[test] Filtered Hits@10: 2.44
INFO:tensorflow:Epoch 2
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:
[<tf.Variable 'e_embed:0' shape=(40943, 200) dtype=float32_ref>,
 <tf.Variable 'p_embed:0' shape=(18, 200) dtype=float32_ref>,
 <tf.Variable 'batch_normalization/gamma:0' shape=(1,) dtype=float32_ref>,
 <tf.Variable 'batch_normalization/beta:0' shape=(1,) dtype=float32_ref>,
 <tf.Variable 'conv2d/kernel:0' shape=(3, 3, 1, 32) dtype=float32_ref>,
 <tf.Variable 'conv2d/bias:0' shape=(32,) dtype=float32_ref>,
 <tf.Variable 'batch_normalization_1/gamma:0' shape=(10368,) dtype=float32_ref>,
 <tf.Variable 'batch_normalization_1/beta:0' shape=(10368,) dtype=float32_ref>,
 <tf.Variable 'dense/kernel:0' shape=(10368, 200) dtype=float32

100%|█████████████████████████████| 5000/5000 [01:40<00:00, 49.61it/s]


[test] Raw MRR: 0.01567496903668097
[test] Raw Hits@1: 0.8
[test] Raw Hits@3: 1.7399999999999998
[test] Raw Hits@5: 1.8399999999999999
[test] Raw Hits@10: 2.76
[test] Filtered MRR: 0.01579012670680101
[test] Filtered Hits@1: 0.8
[test] Filtered Hits@3: 1.7399999999999998
[test] Filtered Hits@5: 1.8399999999999999
[test] Filtered Hits@10: 2.82
INFO:tensorflow:Epoch 3
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:
[<tf.Variable 'e_embed:0' shape=(40943, 200) dtype=float32_ref>,
 <tf.Variable 'p_embed:0' shape=(18, 200) dtype=float32_ref>,
 <tf.Variable 'batch_normalization/gamma:0' shape=(1,) dtype=float32_ref>,
 <tf.Variable 'batch_normalization/beta:0' shape=(1,) dtype=float32_ref>,
 <tf.Variable 'conv2d/kernel:0' shape=(3, 3, 1, 32) dtype=float32_ref>,
 <tf.Variable 'conv2d/bias:0' shape=(32,) dtype=float32_ref>,
 <tf.Variable 'batch_normalization_1/gamma:0' shape=(10368,) dtype=float32_ref>,
 <tf.Variable 'batch_normalization_1/beta:0' shape=(10368,) dtype=float32_ref>,
 <tf.Vari

...

[test] Raw MRR: 0.49875417684733986
[test] Raw Hits@1: 37.62
[test] Raw Hits@3: 56.82000000000001
[test] Raw Hits@5: 64.74
[test] Raw Hits@10: 73.5
[test] Filtered MRR: 0.671207172540884
[test] Filtered Hits@1: 57.92
[test] Filtered Hits@3: 73.3
[test] Filtered Hits@5: 78.12
[test] Filtered Hits@10: 83.3
INFO:tensorflow:Epoch 99
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:
[<tf.Variable 'e_embed:0' shape=(40943, 200) dtype=float32_ref>,
 <tf.Variable 'p_embed:0' shape=(18, 200) dtype=float32_ref>,
 <tf.Variable 'batch_normalization/gamma:0' shape=(1,) dtype=float32_ref>,
 <tf.Variable 'batch_normalization/beta:0' shape=(1,) dtype=float32_ref>,
 <tf.Variable 'conv2d/kernel:0' shape=(3, 3, 1, 32) dtype=float32_ref>,
 <tf.Variable 'conv2d/bias:0' shape=(32,) dtype=float32_ref>,
 <tf.Variable 'batch_normalization_1/gamma:0' shape=(10368,) dtype=float32_ref>,
 <tf.Variable 'batch_normalization_1/beta:0' shape=(10368,) dtype=float32_ref>,
 <tf.Variable 'dense/kernel:0' shape=(10368, 20

100%|█████████████████████████████| 5000/5000 [01:40<00:00, 49.70it/s]


[test] Raw MRR: 0.49897352283582513
[test] Raw Hits@1: 37.64
[test] Raw Hits@3: 57.14
[test] Raw Hits@5: 64.4
[test] Raw Hits@10: 73.38
[test] Filtered MRR: 0.6679563070078575
[test] Filtered Hits@1: 57.720000000000006
[test] Filtered Hits@3: 72.61999999999999
[test] Filtered Hits@5: 77.60000000000001
[test] Filtered Hits@10: 82.94
INFO:tensorflow:Epoch 100
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:
[<tf.Variable 'e_embed:0' shape=(40943, 200) dtype=float32_ref>,
 <tf.Variable 'p_embed:0' shape=(18, 200) dtype=float32_ref>,
 <tf.Variable 'batch_normalization/gamma:0' shape=(1,) dtype=float32_ref>,
 <tf.Variable 'batch_normalization/beta:0' shape=(1,) dtype=float32_ref>,
 <tf.Variable 'conv2d/kernel:0' shape=(3, 3, 1, 32) dtype=float32_ref>,
 <tf.Variable 'conv2d/bias:0' shape=(32,) dtype=float32_ref>,
 <tf.Variable 'batch_normalization_1/gamma:0' shape=(10368,) dtype=float32_ref>,
 <tf.Variable 'batch_normalization_1/beta:0' shape=(10368,) dtype=float32_ref>,
 <tf.Variable 'den

100%|█████████████████████████████| 5000/5000 [01:40<00:00, 49.56it/s]


[test] Raw MRR: 0.49699243419396805
[test] Raw Hits@1: 37.36
[test] Raw Hits@3: 57.14
[test] Raw Hits@5: 64.60000000000001
[test] Raw Hits@10: 73.38
[test] Filtered MRR: 0.6681777772533513
[test] Filtered Hits@1: 57.68
[test] Filtered Hits@3: 72.86
[test] Filtered Hits@5: 77.66
[test] Filtered Hits@10: 83.04
